In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pickle


In [6]:
df = pd.read_csv(r"C:\Users\adminS\OneDrive\Data Science\hauntify_posts.csv")
print("First 5 rows of dataset:")
df.head(100)
df.shape

First 5 rows of dataset:


(108, 2)

Convert text to numbers (TF-IDF)

In [7]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['post'])
y = df['label']


Split data into training and testing

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [9]:
#Train the model
model = LogisticRegression()
model.fit(X_train, y_train)


LogisticRegression()

In [11]:
#Test the model
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))



Accuracy: 0.9090909090909091
              precision    recall  f1-score   support

           0       0.82      1.00      0.90        14
           1       1.00      0.84      0.91        19

    accuracy                           0.91        33
   macro avg       0.91      0.92      0.91        33
weighted avg       0.93      0.91      0.91        33



In [13]:
#Predict spookiness for a new post
def predict_spookiness(post):
    text_vec = vectorizer.transform([post])
    prediction = model.predict(text_vec)[0]
    score = model.predict_proba(text_vec)[0][1] * 100  # Probability of spookiness
    label = "🩸 Spooky!" if prediction == 1 else "🎃 Not spooky"
    return f"{label} (Spookiness Score: {score:.2f}%)"

# Example
print(predict_spookiness("The ghost whispered my name..."))
print(predict_spookiness("Had fun at the costume party!"))


🩸 Spooky! (Spookiness Score: 56.76%)
🎃 Not spooky (Spookiness Score: 43.58%)


In [14]:
#Recommend costumes
def recommend_costume(score):
    if score > 80:
        return "👻 Ghost, 💀 Skeleton, 🧛 Vampire, 🧙 Witch"
    elif score > 60:
        return "🧟 Zombie, 🧞 Genie, 🧝 Elf"
    elif score > 40:
        return "🧚 Fairy, 🐺 Werewolf, 🦹 Villain"
    else:
        return "🎩 Magician, 🐱 Cat, 🎃 Pumpkin"

# Example
post = "Something moved in the dark behind me..."
text_vec = vectorizer.transform([post])
score = model.predict_proba(text_vec)[0][1] * 100
print("Spookiness Score:", round(score,2))
print("Recommended Costumes:", recommend_costume(score))


Spookiness Score: 73.28
Recommended Costumes: 🧟 Zombie, 🧞 Genie, 🧝 Elf


In [15]:
# Saving the model
with open('sentiment_model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

print("Model and vectorizer saved!")


Model and vectorizer saved!


In [ ]:
# streamlit_app.py
import streamlit as st
import pickle
import cv2
from PIL import Image
import numpy as np

# Load saved model and vectorizer
with open('sentiment_model.pkl', 'rb') as f:
    model = pickle.load(f)
with open('tfidf_vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

# ----------------------------
# Step 1: Spookiness prediction
# ----------------------------
def predict_spookiness(post):
    vec = vectorizer.transform([post])
    pred = model.predict(vec)[0]
    score = model.predict_proba(vec)[0][1] * 100
    label = "🩸 Spooky!" if pred == 1 else "🎃 Not spooky"
    return label, score

# ----------------------------
# Step 2: Costume recommendation
# ----------------------------
def recommend_costume(score):
    if score > 80:
        return "👻 Ghost, 💀 Skeleton, 🧛 Vampire, 🧙 Witch"
    elif score > 60:
        return "🧟 Zombie, 🧞 Genie, 🧝 Elf"
    elif score > 40:
        return "🧚 Fairy, 🐺 Werewolf, 🦹 Villain"
    else:
        return "🎩 Magician, 🐱 Cat, 🎃 Pumpkin"

# ----------------------------
# Step 3: Facial makeup spookiness
# ----------------------------
def rate_makeup(image):
    img = np.array(image.convert('RGB'))
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    if len(faces) == 0:
        return "No face detected"
    scores = []
    for (x, y, w, h) in faces:
        face = img[y:y+h, x:x+w]
        brightness = face.mean()
        contrast = face.std()
        # simple "blood/red" detection
        r,g,b = cv2.split(face)
        red_mask = (r>120) & (r>g+30) & (r>b+30)
        red_ratio = red_mask.sum() / (w*h)
        score = (contrast/128)*40 + (1 - brightness/255)*40 + (red_ratio*100)*0.2
        scores.append(np.clip(score, 0, 100))
    return round(np.mean(scores),2)

# ----------------------------
# Streamlit Interface
# ----------------------------
st.title("Hauntify 🎃 — Spookiness & Costume Recommender")

post = st.text_area("Enter your Halloween post:")

uploaded_file = st.file_uploader("Upload your face image (optional)", type=["jpg","png","jpeg"])

if st.button("Analyze"):
    if post:
        label, score = predict_spookiness(post)
        st.metric("Spookiness", f"{score:.2f}% ({label})")
        st.subheader("Recommended Costumes")
        st.write(recommend_costume(score))
    if uploaded_file:
        image = Image.open(uploaded_file)
        st.image(image, caption="Uploaded Image", width=250)
        makeup_score = rate_makeup(image)
        st.metric("Makeup Spookiness", f"{makeup_score}")
